In [5]:
import os
 
text_data_directory = "E:/Data/Sb_Translations"
sample_text = text_data_directory + "/43/translated_For translation 2.txt"

print(sample_text)
if os.path.isfile(sample_text):
    print("Файл существует")
else:
    print("Файл не существует")

E:/Data/Sberbank_Translations/43/translated_For translation 2.txt
Файл существует


In [4]:
import pyspark
import pyspark.sql.functions as f

from pyspark.sql.functions import udf 
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer

pattern_punct = '[!@"“’№«»#$%&\'()*+,—/:;<=>?^_`{|}~\[\]]'
pattern_url = 'http[s]?://\S+|www\.\S+'
pattern_tags = '<.*?>'

def text_to_low(text) -> str:
    return str(text).lower()

text_data_directory = "E:/Data/Sb_Translations"
sample_text = text_data_directory + "/43/translated_For translation 2.txt"

spark = SparkSession.builder \
                        .master("local[*]") \
                        .appName("text data") \
                        .getOrCreate()
df = spark.read.format('text').load('file:///' + sample_text)

df.show(2)

# нужно удалить все местоимения
# перевести все в нижний регистр
# удалить всевозможные символы знаки припенания, эмодзи, ссылки

text_to_lower = udf(text_to_low, StringType())

df = df.withColumn('value1', text_to_lower('value'))
df = df.drop('value').withColumnRenamed('value1', 'value')
df = df.withColumn('value', f.regexp_replace('value', pattern_punct, ''))
df = df.withColumn('value', f.regexp_replace('value', pattern_url, ''))
df = df.withColumn('value', f.regexp_replace('value', pattern_tags, ''))

df.show(2)

# токинезация
regexTokenizer = RegexTokenizer(inputCol="value", outputCol="tokens", pattern=r"\s+")
df = regexTokenizer.transform(df)

df.show(5)


+-----------+
|      value|
+-----------+
| Форма 2.2.|
|Опыт работы|
+-----------+
only showing top 2 rows

+-----------+
|      value|
+-----------+
| форма 2.2.|
|опыт работы|
+-----------+
only showing top 2 rows

+--------------------+--------------------+
|               value|              tokens|
+--------------------+--------------------+
|          форма 2.2.|       [форма, 2.2.]|
|         опыт работы|      [опыт, работы]|
|информация о клие...|[информация, о, к...|
|информация о реал...|[информация, о, р...|
|                  пп|                [пп]|
+--------------------+--------------------+
only showing top 5 rows

